<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Primeiro-Passo:-Armazenamento" data-toc-modified-id="Primeiro-Passo:-Armazenamento-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Primeiro Passo: Armazenamento</a></span><ul class="toc-item"><li><span><a href="#Virtual-Private-Cloud" data-toc-modified-id="Virtual-Private-Cloud-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Virtual Private Cloud</a></span></li><li><span><a href="#Sub-redes" data-toc-modified-id="Sub-redes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Sub-redes</a></span></li><li><span><a href="#Internet-Gateway" data-toc-modified-id="Internet-Gateway-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Internet Gateway</a></span></li><li><span><a href="#Tabelas-de-Roteamento" data-toc-modified-id="Tabelas-de-Roteamento-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Tabelas de Roteamento</a></span></li><li><span><a href="#Instância-RDS" data-toc-modified-id="Instância-RDS-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Instância RDS</a></span></li></ul></li></ul></div>

Entre algumas evoluções obtidas em um rico processo de construção e documentação do projeto _NBAFlow_, este notebook possui, como ponta de partida, o resultado obtido a partir da primeira entrega do [Painel de Estatísticas de Jogadores](https://public.tableau.com/app/profile/thiago.henrique.gomes.panini/viz/NBAFlow-InsightsdeDadosdaNBA/PaineldeEstatsticasdeJogadores) já publicado no [Tableau Public](https://public.tableau.com/s/). Dentro deste primeiro produto entregue, é possível consolidar o fluxo dos dados a partir do seguinte diagrama:

<div align="center">
    <br><img src="https://i.imgur.com/HMfAMMV.jpg" alt="nbaflow-c4">
</div>

Em resumo, o painel analítico construído e publicado no Tableau é gerado a partir de uma fonte de dados, concebida em formato csv e armazenada localmente, cuja construção tem, como base, o processamento de scripts Python capazes de consumir _endpoints_ presentes na biblioteca `nba_api`. Neste primeiro momento, a solução adotada atendia perfeitamente os requisitos estabelecidos, permitindo assim uma rápida construção de um produto relevante.

Em sequência ao desenvolvimento inicial proposto, o objetivo deste notebook é dar início aos passos necessários a serem concebidos visando um aprimoramento na arquitetura existente, abrindo margem para armazenamento e processamento dos dados em nuvem através da [AWS](https://aws.amazon.com/pt/). Conceitos e serviços como VPC, subredes, RDS, EC2, S3, SNS, entre outros, podem eventualmente serem abordados neste material. O intuito é explorar algumas possibilidades previamente estabelecidas e, ao longo do desenvolvimento, mapear novas ideias que possam surgir.

___
**_Para entender todo o contexto do projeto, é sugerido o consumo dos seguintes materiais:_**

* [1. NBAFlow - documentação e exploração inicial da API nba_api](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-doc-api.ipynb)

* [2. NBAFlow - documentação de fluxo de extração de imagens de jogadores](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-players-images.ipynb)

* [3. NBAFlow - estruturação de armazenamento em banco de dados Postgres (OPCIONAL)](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-psql.ipynb)

* [4. Classes estruturadas de processamento de dados](https://github.com/ThiagoPanini/nbaflow/tree/main/core)

* [5. Scripts de processamento de dados utilizando as classes estruturadas](https://github.com/ThiagoPanini/nbaflow/tree/main/scripts)
___

# Primeiro Passo: Armazenamento

Como mencionado anteriormente, a primeira solução do _dashboard analítico_ do projeto NBAFlow tem, como fonte de dados, um arquivo `.csv` gerado a partir da execução de scripts Python consturídos justamentepara este fim. Como uma primeira tentativa de contato com serviços da AWS, é possível analisar as possibilidades de armazenar estes dados em um serviço de banco de dados na nuvem. Dentro da construção do arquivo e, considerando seu conteúdo, o serviço da AWS que mais se enquadra neste cenário é o banco de dados relacional [RDS](https://aws.amazon.com/pt/rds/) (do inglês, _Relational Database Service_). Da página oficial do serviço, temos:

> _O Amazon Relational Database Service (Amazon RDS) facilita a configuração, a operação e a escalabilidade de bancos de dados relacionais na nuvem. O serviço oferece capacidade econômica e redimensionável e automatiza tarefas demoradas de administração, como provisionamento de hardware, configuração de bancos de dados, aplicação de patches e backups. Dessa forma, você pode se concentrar na performance rápida, alta disponibilidade, segurança e conformidade que os aplicativos precisam._
_O Amazon RDS está disponível em vários tipos de instância de banco de dados – com otimização para memória, performance ou E/S – e oferece seis mecanismos de bancos de dados comuns, incluindo Amazon Aurora, PostgreSQL, MySQL, MariaDB, Oracle Database e SQL Server. Você pode usar o AWS Database Migration Service para migrar ou replicar facilmente bancos de dados existentes para o Amazon RDS._

Como uma forma estrutural de propor a construção e a implantação de serviços da AWS em um ambiente isolado capaz de fornecer toda a suite de conectividade exigida pelo projeto, os próximos passos dessa seção de armazenamento serão pautados pela organização de redes e grupos de acessos pertinentes à proposta em questão. Assim, criações de novas [VPCs](https://aws.amazon.com/pt/vpc/) (do inglês, _Virtual Private Cloud_), sub-redes e grupos de segurança serão exemplificados como forma de garantir a estrutura do projeto

## Virtual Private Cloud

Da página do serviço, disponibilizada pela própria AWS, temos:

>_A Amazon Virtual Private Cloud (Amazon VPC) é um serviço que permite iniciar recursos da AWS em uma rede virtual isolada logicamente definida por você. Você tem controle total sobre seu ambiente de redes virtuais, incluindo a seleção do seu próprio intervalo de endereços IP, a criação de sub-redes e a configuração de tabelas de rotas e gateways de rede. Você pode usar IPv4 e IPv6 para a maioria dos recursos em sua nuvem privada virtual, garantindo acesso fácil e seguro a recursos e aplicações._
_Como um dos serviços básicos da AWS, o Amazon VPC facilita a personalização da configuração de rede da VPC. Você pode criar uma sub-rede voltada ao público para seus servidores Web que têm acesso à Internet. Também é possível colocar seus sistemas back-end, como bancos de dados ou servidores de aplicações, em uma sub-rede privada, sem acesso à Internet. Com o Amazon VPC, você pode usar várias camadas de segurança, incluindo grupos de segurança e listas de controle de acesso à rede, para ajudar a controlar o acesso às instâncias do Amazon EC2 em cada sub-rede._

Assim, dentro do objetivo simplório em criar um banco de dados relacional na nuvem capaz de armazenar os dados de jogadores da NBA obtidos a partir da biblioteca `nba_api`, criar uma rede isolada de recursos da provedora cloud é um passo importante para a implantação de recursos e a manutenção da organização de acessos entre os serviços e a internet pública.

Por padrão, a AWS fornece uma VPC já com um bloco [CIDR](https://en.wikipedia.org/wiki/Classless_Inter-Domain_Routing) (do inglês, _Classeless Inter-Domain Route)_ associado. No exemplo abaixo, a VPC padrão `vpc-ea4ab68c` pode ser vista em detalhes, incluindo um parâmetro extremamente importante dentro do entendimento da conectividade interna e externa dos dispositivos internos dessa rede: o bloco de IPs CIDR. Na imagem, a partir do endereço `172.31.0.0/16`, associado ao bloco CIDR da VPC, é possível extrair, por exemplo, informações como:

* Trata-se de um endereço de IP da classe B
    - Primeiro octeto iniciado por 172 está entre 128 e 191, determinando assim o pertencimento à classe B
    - Por ser de classe B, existem $2^{16}=65536$ possibilidades de conexão (redes, hosts e broadcasts)
    - Máscara de sub-rede equivalente a 255.255.0.0 (característica de IPs da classe B)
* A notação CIDR indica que existem diferentes sub-redes 
    - O número 16, após a barra "/", indica que são utilizados 16 bits "1s" na máscara de sub-rede atrelada
    - Transformando em binário, temos que a máscara equivale aos octetos: 11111111.11111111.00000000.00000000
    - Após essa confirmação, de fato, a máscara de sub-rede equivale a 255.255.0.0, mantendo assim as 65536 possibilidades de conexão

<div align="center">
    <br><img src="https://i.imgur.com/dCobYeO.png" alt="aws-defaultvpc">
</div>

Após esse entendimento inicial, será proposta a criação de uma nova VPC para alocação dos recursos de novos projetos dentro da AWS. Considerando um cenário exploratório, essa nova VPC será construída com base em estudos de endereçamento de modo a propor uma arquitetura enxuta e organizada dos recursos nela alocados. Assim, a figura abaixo ilustra a `vpc-dev` dentro de um bloco CIDR definido pela máscara `192.168.0.0/16`, indicando assim um endereço de máscara definido por `255.255.0.0` ou, em binário, `11111111.11111111.00000000.00000000`.

<div align="center">
    <br><img src="https://i.imgur.com/9MOONa6.png" alt="aws-defaultvpc">
</div>

Com isso, é possível seguir na construção da estrutura de rede para a implantação do projeto de acordo com diretrizes organizacionais e boas práticas de utilização de endereçamento. No próximo tópico, será proposto um entendimento inicial e a criação de algumas sub-redes, públicas e privadas, em diferentes zonas de disponibilidade da AWS. Para cada sub-rede, será estabelecido um bloco CIDR de acordo com a capacidade de alocação de dispositivos definida.

## Sub-redes

Observando o bloco CIDR definido para a VPC recém criada (`192.168.0.0/16`), é possível criada tem a capacidade, por si só, de alocar $2^{16}=65534$ conexões (reserva de 1 endereço de rede e 1 endereço de broadcast). Nesse formato, iniciou-se então estudos para a criação de diferentes sub-redes dentro da VPC criada. Visando obter um maior entendimento sobre o endereçamento de redes e a alocação de recursos, foram definidos blocos CIDR variados para as sub-redes criadas em diferentes zonas de disponibilidade. A imagem abaixo traz todas as sub-redes criadas.

<div align="center">
    <br><img src="https://i.imgur.com/ZO4rS1C.png" alt="aws-subnets">
</div>

Observando a imagem acima, é possível destacar o endereçamento de cada bloco CIDR definido para cada sub-rede do projeto. Estrategicamente, as sub-redes foram criadas utilizando uma máscara de sub-rede definida pela sufixo `/24` (`11111111.11111111.11111111.00000000` em binário ou `255.255.255.0` em decimal) e, neste cenário, alterou-se apenas o terceiro octeto como uma fórma de representar o "índice" da sub-rede, facilitando assim a organização e o gerenciamento do endereçamento proposto. Ao fim do processo, definiu-se a criação de sub-redes com seus respectivos endereçamentos definidos por `192.168.X.0/24`, onde X pode ser um número qualquer entre 0 e 255. Dentro da proposta de criação de um espaço de alocação de IPs, público e privado, para duas zonas de disponibilidades diferentes, as sub-redes criadas foram:

    - 192.168.1.0/24
    - 192.168.2.0/24
    - 192.168.3.0/24
    - 192.168.4.0/24

Observando as máscaras definidas para cada sub-rede, percebe-se que o último octeto é o único que varia dentro do intervalo de endereços de IPs possíveis. Sendo assim, cada bloco possui a capacidade para 254 endereços (já retirando o próprio endereço de rede e o broadcast). Na imagem referenciada acima, é possível notar a capacidade para 251 endereços de IPs (a AWS provavelmente reserva mais alguns endereços para outros fins).

Por fim, é este tópico encerra-se com o desenho do diagrama de soluções obtido até o momento, ao qual pode ser visualizado a partir da figura abaixo gerada pelo [draw.io](https://app.diagrams.net/):

<div align="center">
    <br><img src="https://i.imgur.com/liKM1SX.png" alt="aws-diagram-v0">
</div>

Após um entendimento prévio sobre os blocos fundamentais de construção relacionados à etapas de redes, é preciso definir algumas questões voltadas a como tais blocos se relacionam entrem si e através da internet. 

## Internet Gateway

Em um primeiro momento, com o intuito de providenciar conectividade dos recursos implantados nas camadas de rede definidas pela VPC e as sub-redes nela inseridas, é preciso criar um componente capaz de gerenciar esse tráfego: o [Internet Gateway](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Internet_Gateway.html).

> _De acordo com a própria página da AWS, um Internet Gateway nada mais é do que um componente capaz de permitir conexões entre a VPC e a internet, servindo a dois principais propósitos: proporcionar um destino **target** na tabela de roteamento da VPC (ou **route table**, do inglês) e também performar um processo conhecido como **Network Address Translation** (ou NAT) para instâncias que contenham um endereço publico de IPv4._

Assim, através do console de gerenciamento da AWS, foi criado um componente Internet Gateway, sendo este associado a VPC `vpc-dev` recém criada.

<div align="center">
    <br><img src="https://i.imgur.com/1kAMpwm.png" alt="aws-igw">
</div>

Após a criação dessa estrutura, é preciso utilizar um recurso da AWS extremamente importante no que diz respeito ao roteamento e direcionamento do tráfego: as tabelas de roteamento.

## Tabelas de Roteamento

As [tabelas de roteamento](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Route_Tables.html) (ou _route tables_) são instrumentos essenciais para a definição de regras utilizadas no gerenciamento de como o tráfego das sub-redes ou _gateways_ é direcionado. Da página oficial, é possível pontuar que:

> _Uma VPC possui um roteador implícito e é possível utilizar tabelas de roteamento para controlar o direcionamento do tráfego. Cada sub-rede na VPC necessita estar associada a uma tabela de roteamento para o gerenciamento das rotas à ela associda. Também é possível associar explicitamente, e de maneira individual, uma sub-rede a uma tabela de roteamento. Caso contrário, as sub-redes serão associadas a tabela de roteamento principal já existente. Uma sub-rede somente pode estar associada a uma tabela de roteamento por vez, porém é possível associar múltiplas sub-redes a uma mesma tabela. 
Opcionalmente, é possível associar uma tabela de roteamento a um internet gateway ou a um vpn gateway, possibilitando assim a especificação regras para tráfego de entrada na VPC._

Neste cenário, uma _route table_ serve como um controlador de roteamento de tráfego, sendo este associado a uma ou mais sub-redes. A partir deste instrumento, é possível gerenciar ou habilitar rotas de acordo com as necessidades do projeto. Observando a definição teórica na documentação do componente _internet gateway_, é possível encontrar tópicos relevantes sobre situações práticas envolvendo os conceitos analisados até o presente momento. Como exemplo, para habilitar acesso a internet através de recursos implantados na AWS, é possível:

> 1. Criar um internet gateway e anexá-lo a uma VPC
2. Adicionar uma rota na _route table_ da sub-rede presente na VPC em questão para direcionar o tráfego de entrada para o componente internet gateway
3. Garantir que instâncias implantadas nesta sub-rede possuam um endereço de IP global único (IPv4 público, Elastic IP ou IPv6)
4. Garantir que as regras contidas no(s) NACL e _security group_ permitam o fluxo estabelecido para a instância na sub-rede

**_Sobre sub-redes públicas e privadas:_**
Se uma sub-rede está associada a uma _route table_ que possui uma rota para um _internet gateway_, então é esta é uma sub-rede **pública**. Se uma sub-rede está associada a uma _route table_ que não possui rota para um _internet gateway_, esta é conhecida como uma sub-rede **privada**.

Dentro da estrutura criada até aqui, temos 4 sub-redes definidas por:
* pub-subnet-1a
* pvt-subnet-1a
* pub-subnet-1b
* pvt-subnet-1b

Considerando a afirmação acima, será proposta a criação de duas tabelas de roteamento considerando as propostas de sub-redes pública e privada definidas após a criação da VPC. Para a dupla de sub-redes públicas, será definido um destino para o _internet gateway_ recém criado. Para a dupla de sub-redes privadas, não será definido um destino para o _igw_. Nesse formato, um exemplo de uma tabela de roteamento para as sub-redes públicas criadas na VPC de desenvolvimento é dado por:

<div align="center">
    <br><img src="https://i.imgur.com/PK3Ssju.png" alt="aws-rtb">
</div>

Por fim, uma nova versão do diagrama da solução pode ser visualizada a partir da figura abaixo:

<div align="center">
    <br><img src="https://i.imgur.com/XpIcTpA.png" alt="aws-diagrama-v1">
</div>

## Instância RDS

Uma vez criada e configurada a estrutura do projeto (com um requinte estratégico de aprendizado), é possível partir para a implantação de um banco de dados relacional da AWS. Também conhecido como RDS (do inglês, _Relational Database Service_), este serviço proporciona o armazenamento de dados em um banco com infraestrutura gerenciada pela própria provedora cloud, permitindo assim aos usuários um maior gerenciamento de recursos de acordo com as provisões necessárias para o projeto. No início desta seção, é possível visualizar um resumo sobre o serviço extraído diretamente de sua [página oficial](https://aws.amazon.com/pt/rds/).

Adentrando a página de criação de um banco RDS na AWS, é possível instanciar um banco _free tier_ a partir de algumas configurações básicas. A imagem abaixo ilustra algumas das etapas de definição e, entre elas, é possível verificar as _engines_ SQL aceitáveis dentro do serviço

<div align="center">
    <br><img src="https://i.imgur.com/LdFgIrc.png" alt="aws-rds-creation">
</div>

Assim, ao finalizar toda a etapa de configuração a partir da seleção dos parâmetros exigidos (entre eles, a zona de disponibilidade a qual a instância EC2 do banco de dados será implementada), tem-se um banco de dados relacional pronto pra uso. Considerando a seleção do _Postgres_ como principal _engine_, os principais atributos deste banco de dados a serem utilizados para conexão são:

* **_endpoint:_** db-nbaflow.c9mwwwoeltyl.sa-east-1.rds.amazonaws.com
* **_username:_** paninit
* **_password:_** \******
* **_db identifier:_** db-nbaflow
* **_porta:_** 5432

Considerando uma atualização no diagrama da solução, temos:

<div align="center">
    <br><img src="https://i.imgur.com/uw4uglM.png" alt="aws-diagram-v2">
</div>

Como próximos passos, será proposta a criação de uma instância EC2 estrategicamente alocada em uma sub-rede acessível ao banco de dados RDS para realizar as primeiras tentativas de conexão.